# DRLND Project 3 Report: Collaboration and Competition
______________________________

## Introduction
This is my submission report for Project 3 (Collaboration & Competition) of Udacity's Deep Reinforcement Learning Nanodegree.  

In this project, we train 2 agents to play against each other in the Udacity Tennis environment. Agents are rewarded for bouncing the ball over the net and punished for knocking it out of bounds or letting it hit the ground. Their overall goal therefore is to keep bouncing the ball to each other for as long as possible. The task has a larger element of collaboration than competition since the only way agents can maximize their score is if their action allows the other agent to hit the ball back, keeping it within grounds. 

### Background to Actor-Critic & DDPG
For my submission, I used DDPG. I covered the background for this algorithm in [Project 2](https://github.com/andrefmsmith/drlnd_ContinuousCtrlSubmission/blob/master/Report.ipynb).  

## DDPG Algorithm
### Overview of my approach
For this assignment, I adapted my implementation of DDPG from the [Continuous Control Project](https://github.com/andrefmsmith/drlnd_ContinuousCtrlSubmission). A step by step description of the general DDPG algorithm has been presented in the [Algorithm overview](https://github.com/andrefmsmith/drlnd_ContinuousCtrlSubmission/blob/master/Report.ipynb) cell block of the Continuous Control Project report.  

For this assignment, I have a single Actor-Critic instance controlling the two agents. The **Actor network** takes current state as input and outputs an action vector, and the **Critic network** takes in current state at the input layer and current action at the first hidden layer, outputting a single action value given current state and action.   

Apart from simplicity, my reasoning for approaching the task with a single Actor-Critic is that both agents are observing similar states and working within the boundaries of the same action space. During task performance, at each timestep the network does a forward pass operation on each of the agents' states and returns an action for each to perform.  

Because of the similarity in state and action spaces, there should be a great degree of generalization occurring in mapping observed states to performed actions between the two rackets.In this environment, I imagine that all of the state and action variables are either:
- Mirrored between agents, ie the net is coordinate 0 and one racket's position is for example +5.0 whereas the other's is -5.0, or
- The same between agents, ie the variables reflect absolute distance to the net (+5.0 in both cases above).  

In the first case, the transform from one agent's state and action space to the others is trivial (a sign inversion) and therefore easy to learn, whereas in the second there's not even any transform needed. In both cases it seems to me we don't necessarily need to learn two policies and have distinct Actor networks controlling the agents and distinct Critic networks evaluating their actions, but instead that a single, wide-enough Actor-Critic should suffice.

### Actor and Critic architectures
The **Actor** is a state-in-action-out network therefore its input layer is sized (state_size, hu[0]), and its output layer is sized (hu[-1], action_size), where hu is a tuple defining the number of hidden units. I went with hu=(256, 128) to ensure the network was wide enough to learn any transforms needed on the state space. The activation function for inner layers was ReLu, whereas for the output layer it was tanh since the action space boundaries were [-1, 1] for both variables. Had it been different, I would have needed to write an appropriate scaling function.  

Finally, for this task I chose to try out Batch Normalization layers, given that trajectories could be quite different.
```Python
class Actor(nn.Module):
    def __init__(self, state_size, action_size, hu=(256, 128),
                 activ_in = F.relu, activ_out = torch.tanh):
        super(Actor, self).__init__()
        
        self.activ_in = activ_in
        self.activ_out = activ_out
        
        self.input_layer = nn.Linear(state_size, hu[0])
        self.bn1 = nn.BatchNorm1d(hu[0])
        self.hl1 = nn.Linear(hu[0], hu[1])
        self.bn2 = nn.BatchNorm1d(hu[1])
        self.output_layer = nn.Linear(hu[-1], action_size)
        self.bn3 = nn.BatchNorm1d(action_size)
        
    def forward(self, state):
        x = state
        x = self.activ_in(self.input_layer(x))
        x = self.bn1(x)
        x = self.activ_in(self.hl1(x))
        x = self.bn2(x)
        x = self.output_layer(x)
        x = self.bn3(x)
        return self.activ_out(x)```  
        
The **Critic** is a state-in-value-out network where its input layer is sized (state_size, hu[0]). The agent's current action is fed in at the first hidden layer by concatenating it to the output of the input layer. The first hidden layer size is therefore (hu[0]+action_size, hu[1]) with the output layer being sized (hu[-1], 1) so that it outputs a single Q value. As done for the Actor, I'm taking advantage of Batch Normalization here as well. The Critic's output is a single float for Q value therefore no activation function is needed:
```Python
class Critic(nn.Module):
    def __init__(self, state_size, action_size, hu=(256, 128), activ_in = F.leaky_relu):
        super(Critic, self).__init__()
        
        self.activ_in = activ_in
        
        self.bn0 = nn.BatchNorm1d(state_size)
        self.input_layer = nn.Linear(state_size, hu[0])
        self.bn1 = nn.BatchNorm1d(hu[0])
        self.hl1 = nn.Linear(hu[0]+action_size, hu[1])
        self.bn2 = nn.BatchNorm1d(hu[1])
        self.output_layer = nn.Linear(hu[-1], 1)
        
    def forward(self, state, action):
        state = self.bn0(state)
        xs = self.input_layer(state)
        xs = self.bn1(xs)
        xs = self.activ_in(xs)
        x = torch.cat((xs, action), dim=1)
        x = self.hl1(x)
        x = self.bn2(x)
        x = self.activ_in(x)
        return self.output_layer(x)```

### DDPG agent Attributes and Methods
I used essentially the same attributes and methods as in the Continuous Control submission. Briefly, the DDPG agent is specified with a number of attributes useful to have easily accessible and then instantiates Online and Target networks for both Actor and Critic, specifying different optimizers for each. This is useful because it works to our advantage to have the Critic learn faster since its output values are used to train the Actor.  

```Python
class DDPG_agent():
    def __init__(self, state_size, action_size, num_agents=2, batch_size=BATCH_SIZE,
                 start_noise=SN, noise_decay=ND, noise_min=NM, add_noise=True, update_cycles=UC):
        super(DDPG_agent, self).__init__()
        self.state_size = state_size
        self.action_size = action_size
        self.num_agents = num_agents
        self.batch_size = batch_size
        
        ### Initialise actor online and target networks
        self.actor_online = Actor(state_size, action_size).to(device)
        self.actor_target = Actor(state_size, action_size).to(device)
        self.actor_optimizer = optim.Adam(self.actor_online.parameters(), lr=LR_ACTOR)
        
        ### Initialise critic online and target networks
        self.critic_online = Critic(state_size, action_size).to(device)
        self.critic_target = Critic(state_size, action_size).to(device)
        self.critic_optimizer = optim.Adam(self.critic_online.parameters(), lr=LR_CRITIC)```

In this assignment I also wrote a function to ensure target and online networks were initialised with the same weights exactly, which sounded like a reasonable idea:
```Python
def equalize_OnlineTarget(self, target, online):
    for target_param, online_param in zip(target.parameters(), online.parameters()):
        target_param.data.copy_(online_param.data)```

My strategy for exploratory behaviour involves simply adding noise drawn from a numpy normal distribution centered at 0 and with standard deviation ```noise_scale``` to the agent's action output, using the method ```generate_noise```. Therefore, the agent is initialised with a certain ```noise_scale```, which is updated by multiplying by a ```noise_decay``` every time the agent takes an action. A noise floor, ```noise_min``` is specified to ensure exploratory behaviour continues throughout the task.  

```Python
### Agent's internal noise-related attributes
        self.noise_scale = start_noise
        self.noise_decay = noise_decay
        self.noise_min = noise_min
        self.add_noise = add_noise
### Agent's method for generating noise
        def generate_noise(self):
            noise = np.random.normal(loc=0, scale=self.noise_scale, size=self.action_size)
            self.noise_scale = max(self.noise_decay*self.noise_scale, self.noise_min)
            return noise```

The agent keeps track of how long it's been since it ran an update with the attribute ```self.update``` and whether enough timesteps have passed to run a new one with hyperparameter ```update_cycles```. As before, the agent has a ```self.memory``` attribute which is an instatiation of the ```ReplayBuffer``` class:

```Python
class ReplayBuffer:
    def __init__(self, action_size, buffer_size, batch_size):
        self.action_size = action_size
        self.buffer_size = buffer_size
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.memory = deque(maxlen=self.buffer_size)

    def add(self, state, action, reward, next_state, done, num_agents):
        """Add a new experience to memory."""
        for i in range(num_agents):
            e = self.experience(state[i], action[i], reward[i], next_state[i], done[i])
            self.memory.append(e)      

    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).float().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)

        return states, actions, rewards, next_states, dones

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)```
        
The only difference to this version being that during execution of the method ```add```, we unpack and add the experiences of each individual agent to memory. This means that later, when we call the method ```learn``` we draw on experiences from both agents to update our networks.  
```Python
def learn(self, experiences, gamma):
        states, actions, rewards, next_states, dones = experiences
        
        actions_next = self.actor_target(next_states)
        Q_targets_next = self.critic_target(next_states, actions_next)
        
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))
        
        Q_expected = self.critic_online(states, actions)
        critic_loss = F.mse_loss(Q_expected, Q_targets)
        
        self.critic_optimizer.zero_grad()
        critic_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.critic_online.parameters(), 1)
        self.critic_optimizer.step()
        
        
        actions_pred = self.actor_online(states)
        actor_loss = -self.critic_online(states, actions_pred).mean()
        
        self.actor_optimizer.zero_grad()
        actor_loss.backward()
        self.actor_optimizer.step()
        
        self.soft_update(self.critic_online, self.critic_target, TAU)
        self.soft_update(self.actor_online, self.actor_target, TAU)```
        
```learn``` takes in a batch of experiences which is sampled randomly by the ```sample``` method of the Replay Buffer class. It queries the target networks first for the action to perform in the next states, and then for the value of these actions, discounting them and adding observed reward in order to return an estimate of the value for the current action given the current state. Then it computes the online Critic's estimate of the current state-action value and the loss, before backpropagation and weight updating.  

To update the Actor, we first run a forward pass through the online Actor network of the batch's states, obtaining the current policy's proposed action. Our 'Loss' is calculated by passing these actions and the states in the batch through the online critic network, which returns their value. If we take its negative, we can perform gradient ascent, thus updating weights in the Actor online network to maximise the likelihood of outputting actions of high value. Finally, both target networks are updated via a weighted average between their parameters and the current online networks'. This equates not to freezing/fixing targets but moving them very slowly.

## Hyperparameters

```Python
BUFFER_SIZE = int(1e6)     # replay buffer size
BATCH_SIZE = 1024            # minibatch size
GAMMA = 0.99               # discount factor
TAU = 1e-3                 # for soft update of target parameters
LR_ACTOR = 3e-4            # learning rate of the actor 
LR_CRITIC = 3e-3           # learning rate of the critic
update_every = 20           # number of timesteps after which to run an update
SN = 0.5                   # starting value for additive noise scale (exploratory actions)
ND = 0.999                 # noise decay rate (exploratory actions)
NM = 0.1                  # noise minimum to be maintained (exploratory actions)
UC = 10                     # Number of cycles to run updates for```

I found it beneficial to work with fairly large batch sizes and a higher learning rate, especially for the Critic network. The latter is particularly useful since the sooner we learn how to infer value well, the earlier we start generating better actions. I kept a fairly high minimum noise added for exploratory actions, and used a high (10) number of update cycles to run per batch. 

## Results
Using this implementation, the task was solved in around 1,000 episodes. During training, I increased the criterion to an average score of 0.75 since I wanted to watch agents playing well. 
![plot](Solved.png)  

## Ideas for future research
- Double Q learning: in DDQN, we use a double network for estimating targets in order to solve the problem of 'optimism' in action values.
- Prioritized replay: instead of sampling experiences at random, do so based on how much there is to be learnt from them. This would involve a different loss function, since prioritised experiences would need to be weighted in proportion to how (in)frequent they are so as not to train on a different distribution from the one actually observed by the agent.
- A 'real' multi-agent implementation: as discussed above, my reasoning for having a single Actor-Critic structure controlling both agents was principled on the structure, generalization desirability and collaborative nature of the task. I'd be curious to implement 2 separate policy networks to see if different strategies unfolded for the 2 agents. One way to do this would perhaps be to implement 2 critics where each has access to its states plus the two agents' actions. I think a shared critic in this situation would result in very similar policies developing, hence my preference for distinct critics, which would also entail each agent having its own replay buffer. The idea would be that competition leads to each agent pulling the other's performance up in an 'arms race', as an alternative to the current more collaborative approach. I don't know if this would work, but I like the 'realism' of an agent being able to observe only its own states and the opponent's actions.
- An interesting thing I noticed in my solution and others' is that performance seems to follow a slow time-scale of peaks and valleys during learning. For example, there's a bump in performance between episode 200 and 500, and a valley from there to around 650. This is followed by a bump into a plateau and a higher bump shortly before 1,000 episodes which itself is followed by a drop and a further increase. I wonder where this behaviour comes from. It can't be the exploration rate contributed by additive noise because that is monotonically declining. Beyond some residual inherent instability of deep nets in function approximation, I don't have a particular idea where these slow dynamics come from, but if the reviewer has a suggestion or literature which has dug into it, I'd be really interested to know more.

Thanks for reading!